Estimación del ingreso trimestral por persona

In [1]:
##LIBRERIAS
librerias <- c("tidyverse",
               "zip",
               "readr",
               "srvyr", 
               "utils")

instalar <- rownames(installed.packages())
instalar_librerias <- setdiff(librerias, instalar)

if (length(instalar_librerias) > 0) {
  install.packages(instalar_librerias)
}

lapply(librerias, library, character.only = TRUE)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Adjuntando el paquete: 'zip'


The following objects are masked from 'package:utils':

    unzip, zip



Adjuntando el paquete: 'srvyr'


The following object is masked from 'package:stats':

    filter




[[1]]
 [1] "lubridate" "forcats"   "stringr"   "dplyr"     "purrr"     "readr"    
 [7] "tidyr"     "tibble"    "ggplot2"   "tidyverse" "stats"     "graphics" 
[13] "grDevices" "utils"     "datasets"  "methods"   "base"     

[[2]]
 [1] "zip"       "lubridate" "forcats"   "stringr"   "dplyr"     "purrr"    
 [7] "readr"     "tidyr"     "tibble"    "ggplot2"   "tidyverse" "stats"    
[13] "graphics"  "grDevices" "utils"     "datasets"  "methods"   "base"     

[[3]]
 [1] "zip"       "lubridate" "forcats"   "stringr"   "dplyr"     "purrr"    
 [7] "readr"     "tidyr"     "tibble"    "ggplot2"   "tidyverse" "stats"    
[13] "graphics"  "grDevices" "utils"     "datasets"  "methods"   "base"     

[[4]]
 [1] "srvyr"     "zip"       "lubridate" "forcats"   "stringr"   "dplyr"    
 [7] "purrr"     "readr"     "tidyr"     "tibble"    "ggplot2"   "tidyverse"
[13] "stats"     "graphics"  "grDevices" "utils"     "datasets"  "methods"  
[19] "base"     

[[5]]
 [1] "srvyr"     "zip"       "lubridate" "forcats"   "stringr"   "dplyr"    
 [7] "purrr"     "readr"     "tidyr"     "tibble"    "ggplot2"   "tidyverse"
[13] "stats"     "graphics"  "grDevices" "utils"     "datasets"  "methods"  
[19] "base"

In [11]:
url<- "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_poblacion_csv.zip"
temp_zip <- tempfile(fileext = ".zip")
temp_dir <- tempdir()
download.file(url, temp_zip, mode = "wb")
unzip(temp_zip, exdir = temp_dir)
poblacion_doc <- list.files(temp_dir, pattern ="poblacion.csv", full.names = TRUE, recursive = TRUE)
poblacion_enigh <- read_csv(poblacion_doc, show_col_types = FALSE)|>
#OBTENER CLAVE DE ENTIDAD
mutate(cve_ent = substr(folioviv, 1,2))|>
#FILTRADO PARA LA CIUDAD DE MÉXICO
filter(cve_ent == "09")|> 
#SELECCIÓN DE CAMPOS DE INTERÉS
select(folioviv, foliohog, numren, nivelaprob, gradoaprob,upm, factor, est_dis, edad, cve_ent)|> 
mutate(gradoaprob = as.numeric(gradoaprob))|>
#UNIR CON TABLA DE INGRESOS
left_join(ingreso_enigh,by = c("folioviv","foliohog","numren"))|>
##ESTABLECER LOS AÑOS DE ESCOLARIDAD
mutate(anos_escolaridad = case_when (nivelaprob %in% c("1", "0") ~ 0,
                        #PRIMARIA
                        nivelaprob=="2" ~ gradoaprob, 
                        #SECUNDARIA
                        nivelaprob=="3" ~ gradoaprob+6, 
                        #PREPARATORIA
                        nivelaprob=="4" ~ gradoaprob+9, 
                        #LICENCIATURA O EQUIVALENTE
                        nivelaprob %in% c("5","6","7") ~ gradoaprob+12, 
                        #POSGRADO
                        nivelaprob %in% c("8","9") ~ gradoaprob+17), 
        #INGRESO MENSUAL
        ing_mensual = ing_tri/3)


str(poblacion_enigh)
head(poblacion_enigh)

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


tibble [8,421 × 13] (S3: tbl_df/tbl/data.frame)
 $ folioviv        : chr [1:8421] "0900006901" "0900006901" "0900006901" "0900006901" ...
 $ foliohog        : num [1:8421] 1 1 1 1 1 1 1 1 1 1 ...
 $ numren          : chr [1:8421] "01" "02" "03" "04" ...
 $ nivelaprob      : num [1:8421] 4 4 4 2 1 2 2 3 4 4 ...
 $ gradoaprob      : num [1:8421] 3 3 3 2 2 6 6 3 3 3 ...
 $ upm             : chr [1:8421] "0003071" "0003071" "0003071" "0003071" ...
 $ factor          : num [1:8421] 1578 1578 1578 1578 1578 ...
 $ est_dis         : chr [1:8421] "141" "141" "141" "141" ...
 $ edad            : num [1:8421] 66 30 32 8 5 71 68 46 25 35 ...
 $ cve_ent         : chr [1:8421] "09" "09" "09" "09" ...
 $ ing_tri         : num [1:8421] 3786 17389 17389 1614 1614 ...
 $ anos_escolaridad: num [1:8421] 12 12 12 2 0 6 6 9 12 12 ...
 $ ing_mensual     : num [1:8421] 1262 5796 5796 538 538 ...


folioviv,foliohog,numren,nivelaprob,gradoaprob,upm,factor,est_dis,edad,cve_ent,ing_tri,anos_escolaridad,ing_mensual
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
0900006901,1,01,4,3,0003071,1578,141,66,09,3785.86,12,1261.9533
0900006901,1,02,4,3,0003071,1578,141,30,09,17388.57,12,5796.1900
0900006901,1,03,4,3,0003071,1578,141,32,09,17388.57,12,5796.1900
0900006901,1,04,2,2,0003071,1578,141,8,09,1614.13,2,538.0433
0900006901,1,05,1,2,0003071,1578,141,5,09,1614.13,0,538.0433
0900006902,1,01,2,6,0003071,1578,141,71,09,5678.80,6,1892.9333


In [12]:
##ESTABLECER EL DISEÑO MUESTRAL
dm<-poblacion_enigh|>
as_survey_design(weights = factor,
                id=upm,
                strata=est_dis)

In [13]:
##APROXIMACIÓN DEL INGRESO MENSUAL DE ACUEERDO AL GRADO DE ESCOLARIDAD
graproes<-dm|>
group_by(anos_escolaridad)|>
summarise(igproes=survey_mean(ing_mensual,na.rm = TRUE, vartype = "cv"))
graproes

anos_escolaridad,igproes,igproes_cv
<dbl>,<dbl>,<dbl>
0,1772.8630,0.09573637
1,1697.9391,0.22483675
2,2189.7338,0.11842374
3,2848.3375,0.11748765
4,2222.0659,0.13863855
5,2708.7470,0.16479705
6,4952.2518,0.04269781
7,2449.5479,0.14418367
8,3536.9337,0.08650728
